In [1]:
import torch
from torch.optim import Adam
from torch.nn import functional as F
from torch.utils.data import DataLoader

import sys
sys.path.append("../")

from shared_utils.data import CSVPromptDataset
from shared_utils.load import get_model, get_tokenizer, configs_from_yaml
from shared_utils.generate import generate_text

from early_exit.patching import replace_attention_layers, set_transformer_early_exit_mode

import wandb
import pandas as pd
import numpy as np

In [2]:
# LOAD IN EXPERIMENT ARGS
# num_epoch = 1                     # args.num_epoch
num_exit_samples = 1                  # args.num_exit_samples
device = "cuda"                    # args.device
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"                    # args.model_name
model_config_path = "../config_deepseek.yaml"                     # args.model_config_path
dataset_path = "../results_and_data/early_exit_sft_dataset/test/data.csv"                  # args.dataset_path
prompt_config_path = "../results_and_data/early_exit_sft_dataset/test/prompt_config.json"                    # args.prompt_config_path
batch_size = 1                    # args.batch_size -- might want to sort out batching, but increasing num_exit_samples might be better + less effort


In [3]:
# LOAD IN THE MODEL AND TOKENIZER
tokenizer = get_tokenizer(model_name)
config = configs_from_yaml(model_config_path, tokenizer.eos_token_id)
model = get_model(model_name, config['model'], device)


# LOAD IN DATASET
dataset = CSVPromptDataset(dataset_path, prompt_config_path)
dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=dataset.collate_fn, shuffle=True)


# ENABLE EARLY EXITING
model = replace_attention_layers(model, config['lora'], device)

replacing layer model.layers.0
replacing layer model.layers.5
replacing layer model.layers.10
replacing layer model.layers.15
replacing layer model.layers.20
replacing layer model.layers.25
address this hack!
trainable params: 2,179,072 || all params: 1,779,276,294 || trainable%: 0.1225


## Testing SFT teacher

In [20]:
class ActivationCollector:
    """
    A utility class to register a forward hook on a specific PyTorch module
    and collect its output during the forward pass.
    
    Attributes:
        activations (torch.Tensor | None): Stores the detached output tensor 
                                           from the hooked layer. It is moved
                                           to the CPU to save GPU memory.
        hook_handle (torch.utils.hooks.RemovableHandle | None): A handle to the
                                                                registered hook,
                                                                used for removal.
    """
    def __init__(self):
        """Initializes the collector."""
        self.activations = []
        self.hook_handle = None

    def _hook_fn(self, module, input_tensors, output_tensor):
        """
        The actual hook function that PyTorch will call. It saves the output
        of the layer.
        """
        # We detach the tensor and move it to the CPU to avoid holding onto
        # the computation graph and to free up GPU memory.
        self.activations.append(output_tensor[0].detach().cpu())
        

    def register(self, model, layer_path: str):
        """
        Registers the forward hook to a specific layer within the model.

        Args:
            model (nn.Module): The model containing the target layer.
            layer_path (str): A dot-separated string path to the target layer
                              (e.g., 'base_model.model.model.norm').
        
        Returns:
            bool: True if registration was successful, False otherwise.
        """
        # First, remove any existing hook managed by this instance.
        self.remove()

        # Navigate through the model hierarchy to find the target layer.
        try:
            target_layer = model
            for part in layer_path.split('.'):
                target_layer = getattr(target_layer, part)
        except AttributeError:
            print(f"Error: Could not find the layer at path: {layer_path}")
            print("Please ensure the path matches the model architecture.")
            return False

        # Register the forward hook on the found layer.
        self.hook_handle = target_layer.register_forward_hook(self._hook_fn)
        print(f"Successfully registered hook on: '{type(target_layer).__name__}' at path '{layer_path}'")
        return True

    def remove(self):
        """
        Removes the registered hook if it exists. It's important to call this
        when you're done to prevent memory leaks.
        """
        if self.hook_handle:
            self.hook_handle.remove()
            self.hook_handle = None
            print("Hook has been removed.")



collector = ActivationCollector()
layer_path_to_hook = 'base_model.model.model.layers.25'

#layer_path_to_hook = 'base_model.model.model.norm'
collector.register(model, layer_path_to_hook)



Successfully registered hook on: 'DynamicallyTypedLayerWithExit' at path 'base_model.model.model.layers.25'


True

In [19]:
collector.remove() 

Hook has been removed.


In [12]:
#model.base_model.model.model.layers

In [21]:
prompt = "Explain the concept of recursion in programming."
system_prompt = "You are a helpful programming tutor."
prefiller = ""

set_transformer_early_exit_mode(model, 'sft_teacher')

with torch.no_grad():
    sft_teacher_response, (sft_teacher_generated_tokens, 
                          sft_teacher_final_layer_logprobs, 
                          gathered_early_exit_hidden_states) = generate_text(
        model=model,
        prompt=prompt,
        system_prompt=system_prompt,
        prefiller=prefiller,
        tokenizer=tokenizer,
        generation_config=config['generation'],
        device=device
    )
    
   # early_output_log_probs = model.early_exit_hidden_state_readout(gathered_early_exit_hidden_states)
    
   # early_exit_probs = model.early_exit_target_probs(
   #     early_output_log_probs=early_output_log_probs,
   #     teacher_final_layer_log_probs=sft_teacher_final_layer_logprobs
   # )

transform_conversations currently only for Deepseek models!
full_tokenize currently only for Deepseek models!
prompt tokens shape: torch.Size([1, 20])


In [13]:
output = model.early_exit_hidden_state_readout(gathered_early_exit_hidden_states[:,-1])
output.shape

torch.Size([1, 399, 151936])

In [22]:
output = torch.cat(collector.activations, dim = 1).cuda()
output.shape

torch.Size([1, 419, 1536])

In [23]:
token_ids=  model.early_exit_hidden_state_readout(output).argmax(-1)

tokenizer.decode(token_ids[0])

"напримернапримернапример � � �/wp �например �FiledFiled регFiled � zug � �<think>Sure � so I need to explain � in programming. � � let've hearing about递 somewhere somewhere but I'm not entirely sure how it exactly is. let me think about \n\n think it refers something nothing nothing with递 calls themselves递 right I'm not entirely100% clear about \n\n it should break it down step-by step.\n\nFirstly what refers something technique used a function calls a problem by breaking it down into smaller versions simpler versions of the same problem. That instead instead's like breaking recursive manoe that calls something递 then calls itself with smaller smaller version of the problem until � sounds sense—it kind how'm not entirely how exactly works exactly.\n\n � me try to举例 how simple example. Like something like calculating factorial factorial of a number. I factorial of n number n is n multiplied by the factorial of nn minus1). So, recursively I递 factorial function factorial(n), it would递 n m

In [23]:
token_ids = sft_teacher_final_layer_logprobs.argmax(-1)
tokenizer.decode(token_ids[0])

", so I need to explain recursion concept of recursion in programming. Hmm, recursion is I remember it's when a function calls itself. But wait, how does that work work? Let me think. an simple example. Maybe something factorial function? Yeah, because's a common one.\n\nSo, factorial of a number n is n multiplied by the of (n-1). That makes sense. But I try to compute factorial3!, it's be 3 * 2!, and 2! is 2 * 1!, and 1! is 1. So, it3! = be 3 * 2 * 1, which is 6. That works straightforward.\n\nBut how does recursion function know the base case? Oh right, the base case is when n is 0 or 1, and then the factorial is 1. So, the the function calls, the function keeps reducing the problem size until it hits the base case.\n\nWait, but each relies on itself. Like, each call the function calls itself with a smaller input. That's the key difference It I'm, are all problem be solved recursively recursion? I think some problems are easier to handle recursively, like those that have overlapping 

In [14]:
token_ids = output.argmax(-1)
token_ids.shape

torch.Size([1, 399])

In [17]:
tokenizer.decode(token_ids[0])

", � stainless need to explain递 concept of递 in programming. � воз starting sounds �'ve vaguely refers something something function refers itself recursively That wait � how does that work work? � me think about how simple example.\n\n Like something classic function? That, factorial sounds commonly classic example.\n\nнапример factorial factorial(n n number n is equal multiplied by factorial(n(nn minus UNS).Base递 sense recursive Starting someone递递递 factorial factorial factorial it would bestdbool3 multipliedstdboolstdbool factorial which vice2 factorial would(Arrays2 *11!, and finally1 factorial is just1 multiplied So递 recursively3 factorial would be 6 *永不2 * �1 = which is easy6. That works pretty.\n\nBut wait does recursion递 itselfBase base case? Oh right, somewhere base case is when n equals zero zero or something1 because whichever factorial factorial factorial is simply1 regardless Otherwise递递递递 step, we function递 going the problem until until it reaches the base case.\n\n � воз为什么

In [5]:
sft_teacher_response

"<｜Assistant｜> You are a helpful programming tutor.\n<｜User｜> Explain the concept of recursion in programming.\n<｜Assistant｜> \nOkay, so I need to explain the concept of recursion in programming. Hmm, recursion... I remember it's when a function calls itself. But wait, how does that actually work? Let me think about a simple example. Maybe the factorial function? Yeah, that's a common example.\n\nSo, factorial of a number n is n multiplied by factorial of (n-1). That makes sense. If I try to compute 3!, it would be 3 * 2!, and 2! is 2 * 1!, and 1! is 1. So, 3! would be 3 * 2 * 1, which is 6. That seems straightforward.\n\nBut how does the function handle the base case? Oh right, the base case is when n is 0 or 1, because then the factorial is 1. So, in the recursive calls, the function keeps reducing the problem size until it reaches the base case.\n\nWait, so recursion builds on itself? Yeah, each time the function calls itself with a smaller input. That's the key. But I wonder, can e

## Testing SFT student

In [6]:
with torch.no_grad():
    batch, gen_len, elayers = early_exit_probs.shape 
    full_len = sft_teacher_generated_tokens.shape[1]
    repeated_sft_teacher_generated_tokens = sft_teacher_generated_tokens.expand(num_exit_samples * batch, full_len)   
    set_transformer_early_exit_mode(model, 'sft_student')
    
    # Create prescribed exit layer idxs filled with torch.inf (always exit on last layer)
    batch_samples, seq_len = repeated_sft_teacher_generated_tokens.shape
    print("Setting exit layers to inf for sft_student")
    prescribed_exit_layer_idxs = torch.full((batch_samples, gen_len), torch.inf, \
                                            device=repeated_sft_teacher_generated_tokens.device)
    print(f"Minimum in prescribed_exit_layer_idxs = {torch.min(prescribed_exit_layer_idxs)}")
    sft_student_output_scores, collected_exit_logits = model(repeated_sft_teacher_generated_tokens,\
                                                             prescribed_exit_layer_idxs=prescribed_exit_layer_idxs)

Setting exit layers to inf for sft_student
Minimum in prescribed_exit_layer_idxs = inf


In [7]:
with torch.no_grad():
    print('CRUDE KL AND MAKE SURE PROBS ARE ALIGNED')
    eps = 1e-16
    sft_teacher_probs = sft_teacher_final_layer_logprobs.softmax(-1)                        # [batch * samples, gen len, vocabulary]
    sft_student_probs = sft_student_output_scores.logits[:,-gen_len:].softmax(-1)           # [batch * samples, gen len, vocabulary]
    token_logits_kl_div = (sft_student_probs * ((sft_student_probs + eps) / (sft_teacher_probs + eps)).log()).sum(-1)   # [batch * samples, gen len]
    
    mean_logit_kl = token_logits_kl_div.mean()

mean_logit_kl

CRUDE KL AND MAKE SURE PROBS ARE ALIGNED


tensor(30.9266, device='cuda:0')

In [8]:
import pandas as pd
from IPython.display import display, Markdown, HTML

def topk_to_df(prob_dist, tokenizer=None, k=5, title="Top-K Predictions"):
    """
    Return top-k predictions and probabilities as a pandas DataFrame.
    """
    top_values, top_indices = torch.topk(prob_dist, k=k)
    
    rows = []
    for i, (idx, prob) in enumerate(zip(top_indices, top_values)):
        token_id = idx.item()
        prob_val = prob.item()
        token_str = tokenizer.decode([token_id]) if tokenizer else str(token_id)
        token_str = repr(token_str)  # Shows escape characters properly
        
        rows.append({
            "Token ID": token_id,
            "Token String": token_str,
            "Probability": prob_val,
        })
    
    df = pd.DataFrame(rows)
    return title, df.round(4)

# Example usage for your loop
dfs = []
for idx in range(5, 11):
    title, df = topk_to_df(sft_student_probs[0, idx], tokenizer, k=5, title=f"Student NTP for token {idx}")
    dfs.append((title, df))

# Display in a grid
html = "<div style='display: flex; flex-wrap: wrap;'>"
for title, df in dfs:
    html += "<div style='flex: 1; min-width: 300px; padding: 10px;'>"
    html += f"<h4>{title}</h4>"
    html += df.to_html(index=False)
    html += "</div>"
html += "</div>"

display(HTML(html))


Token ID,Token String,Probability
11,"','",0.0861
432,' it',0.0620
13,'.',0.0554
279,' the',0.0331
438,' as',0.0303
Token ID,Token String,Probability
279,' the',0.2598
13,'.',0.0589
432,' it',0.0363
1172,' only',0.0192


### Very similar (and gibberish) next token predictions for all tokens. Something wrong!

## Testing free generation

In [9]:
# LOAD IN THE MODEL AND TOKENIZER
tokenizer = get_tokenizer(model_name)
config = configs_from_yaml(model_config_path, tokenizer.eos_token_id)
config['generation']['use_cache'] = False
model = get_model(model_name, config['model'], device)


# ENABLE EARLY EXITING
model = replace_attention_layers(model, config['lora'], device)

replacing layer model.layers.0
replacing layer model.layers.5
replacing layer model.layers.10
replacing layer model.layers.15
replacing layer model.layers.20
replacing layer model.layers.25
address this hack!
trainable params: 2,179,072 || all params: 1,779,276,294 || trainable%: 0.1225


In [10]:
prompt = "Explain the concept of recursion in programming."
system_prompt = "You are a helpful programming tutor."
prefiller = ""

set_transformer_early_exit_mode(model, 'free_generate')

with torch.no_grad():
    free_generate_response, _ = generate_text(
        model=model,
        prompt=prompt,
        system_prompt=system_prompt,
        prefiller=prefiller,
        tokenizer=tokenizer,
        generation_config=config['generation'],
        device=device
    )

transform_conversations currently only for Deepseek models!
full_tokenize currently only for Deepseek models!
prompt tokens shape: torch.Size([1, 20])


In [11]:
free_generate_response

"<｜begin▁of▁sentence｜><｜Assistant｜> You are a helpful programming tutor.\n<｜User｜> Explain the concept of recursion in programming.\n<｜Assistant｜> \nOkay, so I need to explain recursion in programming. Hmm, I remember recursion from my computer science class. It's when a function calls itself repeatedly until a base case is met. That makes sense, but I want to make sure I understand it thoroughly.\n\nLet me think about how it works. When you call a function recursively, it's like solving a problem by breaking it down into smaller sub-problems. Each time the function calls itself, it's handling a smaller part of the problem. The base case is crucial because it's the stopping point. Without a base case, the function would keep calling itself indefinitely, causing a stack overflow error.\n\nWait, how do I identify the base case? It's the simplest version of the problem that can be solved without further recursion. For example, if I have a function that calculates the factorial of a number

## Current status: SFT teacher seems to work, free generation perhaps works, and student does not work.